In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [0]:
import json
import pandas as pd
data_train = [json.loads(line) for line in open('gdrive/My Drive/springer-train.jsonl', 'r')]
df_train=pd.DataFrame(data_train)
df1=pd.concat([df_train.abstract,df_train.title,df_train.keywords,df_train.journal],axis=1)

data_valid = [json.loads(line) for line in open('gdrive/My Drive/springer-valid.jsonl', 'r')]
df_valid=pd.DataFrame(data_valid)
df2=pd.concat([df_valid.abstract,df_valid.title,df_valid.keywords,df_valid.journal],axis=1)

df_new_train=df1.dropna(subset=["abstract"])
df_new_valid=df2.dropna(subset=["abstract"])


In [0]:
df1=pd.DataFrame({'Abstract': list(df_new_train.abstract),
                  'Title': list(df_new_train.title),
                  'Key': list(df_new_train.keywords),
                  'category': list(df_new_train.journal)})
df2=pd.DataFrame({'Abstract': list(df_new_valid.abstract),
                  'Title': list(df_new_valid.title),
                  'Key': list(df_new_valid.keywords),
                  'category': list(df_new_valid.journal)})

In [5]:
import re
import string
import nltk

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import nltk
nltk.download('stopwords')
def preprocess_abstract(text):
    

    text =re.sub("\$\$.*?\$\$", "", text)
    text =re.sub(r"(https.*)", "", text)
    text =re.sub("\$.*?\$", "", text)
    text =re.sub("\\\\\(.*?\\\\\)", "", text)
    text =re.sub("\\\\\[.*?\\\\\]", "", text)
    text =re.sub("\[.*?\]", "", text)
    text =re.sub("{.*?}", "", text)
    text =re.sub(r"\\begin.*?\\end", "", text)
    text = re.sub("[^\w\s\d{}]d".format(string.punctuation), "", text)
    text = text.lower()
    text = re.sub('[^a-z \-]',"",text)
    text=' '.join([item for item in  re.sub('[^a-z \n\-]', ' ', text).split(' ') if len(item) >= 2])
    
    ## Convert words to lower case and split them
    text = text.split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

for i in range(0,len(df1.Abstract)):
  df1.Abstract[i]=preprocess_abstract(df1.Abstract[i])
for i in range(0,len(df2.Abstract)):
  df2.Abstract[i]=preprocess_abstract(df2.Abstract[i])

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df1.Abstract)

sequences = tokenizer.texts_to_sequences(df1.Abstract)
data = pad_sequences(sequences, maxlen=300)

In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df2.Abstract)

sequences = tokenizer.texts_to_sequences(df2.Abstract)
data_valid = pad_sequences(sequences, maxlen=300)

In [0]:
train_y=df1.category
valid_y=df2.category
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
train_y = onehot_encoder.fit_transform(train_y.to_numpy().reshape(len(train_y), 1))
valid_y = onehot_encoder.transform(valid_y.to_numpy().reshape(len(valid_y), 1))

In [10]:
embeddings_index = dict()
f = open('gdrive/My Drive/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [0]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=300, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(196, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [0]:
model_glove.fit(data, np.array(train_y), validation_data=(data_valid,np.array(valid_y)) , epochs = 2)

Train on 244740 samples, validate on 26323 samples
Epoch 1/2
 61728/244740 [======>.......................] - ETA: 12:32 - loss: 0.0352 - accuracy: 0.9934

In [16]:
print(len(df1.category))

244740
